In [29]:
import pandas as pd
import sqlite3

In [30]:
kwBR=pd.read_csv('SeriesKeywords.csv')
kwMG=pd.read_csv('SeriesKeywordsMg.csv')
conn =sqlite3.connect("indicadores.db")
brasilDf=pd.read_sql_query("select * from brazil_df",conn)
statesDf=pd.read_sql_query("select * from states_df",conn)
conn.close()

In [31]:
#obtendo dados apenas das semanas desejadas dos indicadores
brasilDf=brasilDf.iloc[0:44]
statesDf=statesDf[statesDf['state']=='MG']
statesDf=statesDf.reset_index()
statesDf=statesDf.iloc[0:44]

In [32]:
#reordenando colunas para mais facil visualização
cols=kwMG.columns.tolist()
cols=cols[0:1]+cols[-1:]+cols[1:-1]
kwMG=kwMG[cols]
cols=kwBR.columns.tolist()
cols=cols[0:1]+cols[-1:]+cols[1:-1]
kwBR=kwBR[cols]

In [33]:
#normalizando os dados de indicador a serem usados
brasilDf.loc[:,'new_week_cases']=(brasilDf.loc[:,'new_week_cases']/brasilDf.loc[:,'new_week_cases'].max())*100
brasilDf.loc[:,'new_week_deaths']=(brasilDf.loc[:,'new_week_deaths']/brasilDf.loc[:,'new_week_deaths'].max())*100
statesDf.loc[:,'new_week_cases']=(statesDf.loc[:,'new_week_cases']/statesDf.loc[:,'new_week_cases'].max())*100
statesDf.loc[:,'new_week_deaths']=(statesDf.loc[:,'new_week_deaths']/statesDf.loc[:,'new_week_deaths'].max())*100

In [34]:
#criando a coluna de semanas nas keywords
semanas=[]
for i in range(len(kwBR)):
    semanas.append(i+9)
kwMG['epidemiological_week']=semanas
kwBR['epidemiological_week']=semanas

In [35]:
#extraindo apenas dados necessários das databases
brasilDf=brasilDf.set_index('epidemiological_week')
statesDf=statesDf.set_index('epidemiological_week')
brasilDf=brasilDf[['new_week_cases','new_week_deaths']]
statesDf=statesDf[['new_week_cases','new_week_deaths']]
kwBR=kwBR.set_index('epidemiological_week')
kwMG=kwMG.set_index('epidemiological_week')
kwBR=kwBR.drop('date',axis=1)
kwMG=kwMG.drop('date',axis=1)

In [36]:
#Função que encontra as maxlag correlations (até 14 semanas de lag pois é 1/3 das semanas)
def getMaxCorrelations(kw,ind,cat):
    dic={
        'Keyword' : [],
        'MaxCorrelation' : [],
        'Lag' : []        
    }
    for i in kw:
        maxcorr=-1
        lag=-1
        for j in range(14):
            corr=kw[:-j].reset_index()[i].corr(ind[j:].reset_index()[cat],method='spearman')
            if corr>=maxcorr:
                maxcorr=corr
                lag=j
        dic['Lag'].append(lag)
        dic['MaxCorrelation'].append(maxcorr)
        dic['Keyword'].append(i)
    return dic

In [37]:
lagcorr=pd.DataFrame(getMaxCorrelations(kwBR,brasilDf,'new_week_cases'))
lagcorr=lagcorr.set_index('Keyword')
lagcorr.sort_values(by='MaxCorrelation',ascending=False)[0:20]

,MaxCorrelation,Lag
Keyword,,
pcr,0.945470,2
exame covid,0.938506,2
covid pcr,0.927225,2
pcr exame covid,0.921005,3
teste covid,0.875545,3
reagente igg,0.855255,1
peguei covid,0.851196,2
ivermectina covid como tomar,0.842404,2
tomar ivermectina,0.811837,2


In [38]:
lagcorrd=pd.DataFrame(getMaxCorrelations(kwBR,brasilDf,'new_week_deaths'))
lagcorrd=lagcorrd.set_index('Keyword')
lagcorrd.sort_values(by='MaxCorrelation',ascending=False)[0:20]

,MaxCorrelation,Lag
Keyword,,
igg,0.944776,1
igm,0.942281,1
teste igg,0.939638,3
igg igm,0.939384,1
olfato,0.936526,5
paladar e olfato,0.916207,5
tomar ivermectina,0.914199,2
teste rápido covid,0.910329,2
covid igg,0.906900,1


In [39]:
MGlagcorrd=pd.DataFrame(getMaxCorrelations(kwMG,statesDf,'new_week_deaths'))
MGlagcorrd=MGlagcorrd.set_index('Keyword')
MGlagcorrd.sort_values(by='MaxCorrelation',ascending=False)[0:20]

,MaxCorrelation,Lag
Keyword,,
teste covid,0.947195,5
exame covid,0.930734,5
covid pcr,0.912632,5
covid igg,0.885056,3
pcr,0.853018,3
igm,0.834262,5
igg igm,0.812690,5
teste rápido covid,0.800448,8
tomar ivermectina,0.792483,3


In [40]:
MGlagcorr=pd.DataFrame(getMaxCorrelations(kwMG,statesDf,'new_week_cases'))
MGlagcorr=MGlagcorr.set_index('Keyword')
MGlagcorr.sort_values(by='MaxCorrelation',ascending=False)[0:20]

,MaxCorrelation,Lag
Keyword,,
covid pcr,0.906325,4
teste covid,0.894363,3
exame covid,0.870319,2
pcr,0.868106,2
covid igg,0.813108,1
pcr exame covid,0.798552,3
paladar e olfato,0.788132,4
olfato,0.781223,1
estou com covid,0.779919,1


In [41]:
#função que obtém a correlação a partir de um lag específico
def getSpecificLag(kw,ind,cat,lag=0):
    dic={
        'Keyword' : [],
        'Correlation' : [],
        'Lag' : []
    }
    for i in kw:
        corr=kw[:-lag].reset_index()[i].corr(ind[lag:].reset_index()[cat],method='spearman')
        dic['Correlation'].append(corr)
        dic['Keyword'].append(i)
        dic['Lag'].append(lag)
    return dic

In [ ]:
print('Deseja consultar MG ou BR?')
location=input()
print('Qual indicador?(new_week_cases ou new_week_deaths)')
indicator=input()
print('Qual lag?')
lag=int(input())
if(location.upper()=='BR'):
    df=pd.DataFrame(getSpecificLag(kwBR,brasilDf,indicator,lag))
elif(location.upper()=='MG'):
    df=pd.DataFrame(getSpecificLag(kwMG,statesDf,indicator,lag))
df